# Safety
### Unplug power before touching anything
This is what the setup will look like while you are working on it:
![](images/neutral.jpg)
Before you change any of the electrical connections on the setup, you first need to unplugg the power, as shown here:
![](images/disconnected.jpg)
We do this because as we move connections we can accidentally touch other components, and we can also wirde things up the wrong way before we have the time to double check. If the power is on, and you accidentally put 5V to the wrong pin of the Raspberry Pi it will die immediately.  The Raspberry Pi is not the cheap €35 that it used to be, the modeul you are given today is closer to €100. So be careful and triple check your wiring!

### Triple check your wiring with the reference
Each wiring exercise will use a hand-draw circuit diagram. Try to follow this to the best of your ability. Once you are done, before you apply power, compare your setup to one of the images referenced in the exercise.

### If using a multimeter, use extended probe wires
Multimeters are a great tool for trouble shooting electronics, and in this case power has to be on in order to do anything useful. Never mesure directly on a component as seen below:
![](images/dangerous_probes.jpg)
If you slip with the probes a tiny bit you will ned up here:
![](images/failed_probes.jpg)
It doesn't look like much, but in this picture the red probe is connecting the 5V and the 3.3V, and the Raspberry Pi will die immediately. (This exact thing happened to my supervisor)

### Have fun!
All this sounds scary, but failure is a necessary part of learning. If I dind't break mine in the making of the course the chance is low that you will break yours today.

# Raspberry Pi output

[//]: # (Pico: A zoomed out picture when one can see how the red breakout board connects to the Pi itself would be great!)

Here we will look at an introduction to what the Raspberry Pi GPIO pins are and how the pins can be used to turn external components on/off.

Start by connecting the 5V of the red breakout board to the red rail of the breadboard (Long vertival line marked with a **+**). Connect the ground of the breakout board to the blue rail. Also connect the upper and lower parts of the rail to to each other. We do this so that it is simple to connect new components to 5V and GND

![](images/im1.jpg)


## Controlling LEDs

Following the diagram below, connect a wire to GPIO pin 18 (G18 on the red breakout board) and to anywhere on the board. Then connect an LED and a 220 OHM resitor in series to G18 and to ground. Make sure that you understand [how connections on a breadboard works first](https://learn.sparkfun.com/tutorials/how-to-use-a-breadboard/all).

![](images/diagram_1.jpeg)
Circuit reference image can be found at /images/im2.jpg
<!---![](images/im2.jpg)-->

The code below will 
- Initialize the Raspberry Pi's output pins, called GPIO
- turn ON pin 18 providing 3.3 V to our LED 
- wait for 2 seconds, turns OFF the output
- clean up the GPIO objects.

In [ ]:
# Initialize the output pins enabling 18 as output
import RPi.GPIO as GPIO
import time
import scramble
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)

# set the output of pin 18 to HIGH (3.3V)
GPIO.output(18,GPIO.HIGH)
print("LED is on")

# wait two seconds
time.sleep(2)

# set the ouput of pin 18 to LOW (0V)
GPIO.output(18,GPIO.LOW)
print("LED is off")

# clean up the GPIO objects
GPIO.cleanup()

##
## For advance users... you can make this into a function, pin number and wait can be arguments for instance
##

**Exercise:** Add a second Yellow LED and control it using G17.\
**Exercise:** Changing the code above, make the LED blink 10 times in 10 seconds\
**Exercise:** Make the two LEDs blink one after the other

## Controlling Transistors

Now that we have a good indicator that we can control G18, let's try to control a larger current using the combination of G18 and a transistor.

Instead of the original red LED and resitor going to ground, connect them to the base of our transitor (middle pin). Connect the emittor of the transistor to ground (left pin). In this way, when G18 is ON, the transistor allows current to flow from the collector to ground. Now connect a 220 Ohm resistor and a Blue LED in series with to the collector and 5V.
![](images/diagram_2.jpeg)
Circuit reference image can be found at /images/im4.jpg

<!---![](images/im3.jpg)-->

<!---![](images/im4.jpg)-->

Run the same code as before for the red LED. You can find a copy of the original code below:

In [ ]:
# Initialize the output pins enabling 18 as output
import RPi.GPIO as GPIO
import time
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)

# set the output of pin 18 to HIGH (3.3V)
GPIO.output(18,GPIO.HIGH)
print("LED is on")

# wait two seconds
time.sleep(2)

# set the ouput of pin 18 to LOW (0V)
GPIO.output(18,GPIO.LOW)
print("LED is off")

# clean up the GPIO objects
GPIO.cleanup() 

#
# Alternatively just run the already defined function from before ;-)
#

**Exercise:** Which LED is the brightest, and why?\
**Exercise:** Name one place in your home where this could be usefu\
**Exercise:** Name one place in your research where this could be useful

## Pulse Width Modulation (PWM)
G18 can only be ON or OFF, 3.3V or 0.0V. But G18 has a special feature called Pulse Width Modulation (PWM), where in a time span of 1 second it is ON/OFF a certain number of times for a certain duration. https://en.wikipedia.org/wiki/Pulse-width_modulation

We can vary the width of the ON pulses to vary the power output of G18. We can use this to vary the brightness the LEDs.\
**Exercise:** Can you guess what turning an LED on and OFF very quickly this will do to an LED?

In [ ]:
# Initialize the output pins enabling 18 as output
import RPi.GPIO as GPIO
import time
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)

# create PWM instance with frequency, and
# start PWM of required Duty Cycle
pi_pwm = GPIO.PWM(18,1000)
pi_pwm.start(0)

# Now let's play with the PWM 1 second at a time
pi_pwm.ChangeDutyCycle(100)
time.sleep(1)
pi_pwm.ChangeDutyCycle(75)
time.sleep(1)
pi_pwm.ChangeDutyCycle(50)
time.sleep(1)
pi_pwm.ChangeDutyCycle(25)
time.sleep(1)
pi_pwm.ChangeDutyCycle(10)
time.sleep(1)
pi_pwm.ChangeDutyCycle(0)
time.sleep(1)
pi_pwm.ChangeDutyCycle(10)
time.sleep(1)
pi_pwm.ChangeDutyCycle(25)
time.sleep(1)
pi_pwm.ChangeDutyCycle(50)
time.sleep(1)
pi_pwm.ChangeDutyCycle(75)
time.sleep(1)
pi_pwm.ChangeDutyCycle(100)
time.sleep(1)

# Stop the PWM instance
pi_pwm.stop()

# clean up the GPIO objects
GPIO.cleanup() 

**Exercise 1**: Any ideas how this can be useful?

[//]: # (Maybe you want to program some music lights with this?)

In [ ]:
answer_1 = "¨òĆćđ¾ćđ¾ÿ¾đćċĎĊă¾ĕÿė¾Ēč¾āĆÿČąă¾ĒĆă¾ÿċčēČĒ¾čĄ¾ĎčĕăĐ¾ĎēĒ¾ćČĒč¾ÿ¾ĂăĔćāăÌ¾çČđĒăÿĂ¾čĄ¾ĆÿĔćČą¾ÿČ¾ăĊăāĒĐćāÿĊ¾ċčĒčĐ¨¨íìÍíää¾ÿĒ¾ĄēĊĊ¾đĎăăĂ¾čĐ¾Čč¾đĎăăĂÊ¾îõë¾āÿČ¾Āă¾ēđăĂ¾Ēč¾ĐēČ¾ĒĆă¾ċčĒčĐ¾ÿĒ¾ĂćĄĄăĐăČĒ¾đĎăăĂđ¨"
scramble.unscramble(answer_1,"elk")

# ADC and voltage dividers

Next we want to create a voltage divider between 5V and ground and measure the voltage in the middle. On the other half of the breadboard opposite to your LED and transistor setup, connect a wire to 5V and out some distance on the breadboard, like this:

![](images/im5.jpg)

In series to the 5V wire, connect a 91 kOhm resistor, a NTC thermistor, and ground. Keep the long legs of the NTC, you  will need them later. Connect an open ended cable where the resistor and NTC are connected together. We call this the probe. Connect the probe to analog input 0 (A0) on the ADC. Make sure not to connect it to ground or 3.3V. When you are done, have a look at the reference image at /images/im6.jpg

![](images/diagram_3.jpeg)

<!---![](images/im6.jpg)-->

**Rembember, that if you want to double check something with a hand-held multimeter you must always use extended probes wires as outlined in the safety section at the top of this notebook.**

**Exercise**: With 5V, a 91 kOhm resistor, a probe, and a 10 kOhm NTC (at room temperature). Calculate the "theoretical" voltage at your probe.

The code below will use the ADC hat of your Raspberry Pi. A Raspberry Pi hat is a hardware extension that goes on top of your Pi. An ADC is an "Analog to Digital Coverter", it converts analog signals to digital signals. It allows our Raspberry Pi to read voltages. The code below imports the modules necessary for the ADC hat to run, and it then measures the voltage at ADC channel 0

**WARNING! The ADC hat can only measure up to 3.3V, never plug it directly to 5V**

In [ ]:
import i2c_module
import adc_module

adc = adc_module.ADC()
channel=0
milli=1000
milli_volts = adc.read_voltage(channel)
volts = milli_volts/milli
print(f"Voltage at probe: {volts} V")

**Exercise**: Is there a difference between the measured and calculated value? if so, can you think why or why not?\
**Exercise**: The ADC has a 12-bit precision and can measure between 0 and 3.3 volts. What value should the 91 kOhm resistor have to maximize the measurement range? i.e What resistance should we use inorder to have 1.65 volts over the NTC at room temperature?\
**Exercise**: Copy the code from the PWM example. Take a new wire, plug it in to channel 2 of the ADC hat and measure the voltage between the red LED and its resistor. Change the code so that it prints the voltage at each decrease in LED brightness. What do you expect to se?

## Measuring temperature


To calculate the temperature of the NTC we follow a mathematical function given by the manufacturer. This function requires some parameters.

[//]: # (TODO: add a link to the function from the manufacturer)

In [ ]:
import math

# define parameters
channel = 0
R0 = 10000
b = 3936
Rfx = 91000
voltage = 5.2
adc = adc_module.ADC()

deltav=voltage
Vin=adc.read_voltage(channel)/1000
To=298.0  #To of NTC
Ri=int(R0)*math.exp(-int(b)/To)
Rn=(Vin*int(Rfx))/((deltav)-Vin)
T=int(b)/(math.log(Rn/Ri))-273

print(f"Temperature {T} C")

This is potentially very messy to write down each time we want a temperature, so therefore we create a temperature sensor class, which has a function `read_temperature`. In our script, we can then create a sensor object, and call the `read_temperature` function of this object each time we want a temperature reading. If you are not familiar with classes in programming, this will be a good example to get familiar.

In [ ]:
import math
import i2c_module
import adc_module

class NTC:
    def __init__(self,channel,voltage):
        # Define initial parameters of this one sensor object
        self.channel = channel
        self.R0 = 10000
        self.b = 3936
        self.Rfx = 91000
        self.voltage = voltage
        self.adc = adc_module.ADC()

    def read_temperature(self):
        # Function tor return temperature of this one sensor object
        milli = 1000
        deltav=self.voltage
        Vin=self.adc.read_voltage(self.channel)/milli
        To=298.0  #To of NTC
        Ri=int(self.R0)*math.exp(-int(self.b)/To)
        Rn=(Vin*int(self.Rfx))/((deltav)-Vin)
        T=int(self.b)/(math.log(Rn/Ri))-273
        return round(T,2)

After the class is defined, we create an object of that class, and assign it to a variable name

In [ ]:
# NTC object, channel 0, voltage 5.21
ch = 0
vol = 5.21
sensor_1 = NTC(ch, vol)

We can then very easily read the temperature in one line

In [ ]:
print('Temperature', f"{sensor_1.read_temperature()} C", end='\r')

We can also read out the temperature continuously in a `for` loop. In Jupyter click the stop symbol at the top of the notebook to stop the cell execution.

In [ ]:
import time
for i in range(0,20):
    print('Temperature', f"{sensor_1.read_temperature()} C", end='\r')
    time.sleep(1)

**Exercise**: Add a second voltage divider with a second NTC. Create a second temperature sensor object in the code above and print both the temperatures. Ask someone to dobule check your wiring for this exercise

## Measuring temperature of a component

**Don't forget to unplug the 5V from the voltage divider**. Now bend the NTC so that it touches the resistor of the blue LED. Make sure that the legs of the NTC are not touching each other, this would short circuit the NTC. Dobule check that everything looks OK, then re-insert the 5V.

**Exercise**: What do you think will happen to the temperature when G18 is ON?

The code below will measure the temperature while the PWM of G18 is running at full power. The code looks messy, but it is not very different from the gode from the PWM example and the temperature measurement loop above. The added things at the bottom are there to make sure that things are turned off when the code is finished, or when you abort it.

In [ ]:
# Initialize the output pins enabling 18 as output
import RPi.GPIO as GPIO
import time

GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)

# create PWM instance with frequency, and
# start PWM of required Duty Cycle
pi_pwm = GPIO.PWM(18,1000)
pi_pwm.start(0)

print(f"Initial temperature: {sensor_1.read_temperature()}")

pi_pwm.ChangeDutyCycle(100)

# Do the reading for 20 loops or until Ctrl-C is pressed
try:
    for i in range(0,20):
        print('Temperature', f"{sensor_1.read_temperature()} C", end='\r')
        time.sleep(1)

# Exit gracefully if you stop the script and print temperature one last time
except KeyboardInterrupt:
    print(f"Final temperature: {sensor_1.read_temperature()}")
    pi_pwm.stop()
    sys.exit()

# Exit gracefully and print temperature one last time
print(f"Final temperature: {sensor_1.read_temperature()}")
pi_pwm.stop()
GPIO.cleanup()

#
# Alternatively turn this also into a function
#

Unplugg the 5V to the blue LED. Replace the blue LED and its resistor with a big 55 Ohm resistor. Dobule check your wiring with the image below. Re-connect the 5V, but now to the 55 Ohm resistor.

![](images/im7.jpg)

Run the previous code again, but now with the large, lower Ohm resistor.

**Exercise**: What do you think will be different now?\
**Exercise**: What is the difference? Why?\
**Exercise**: Why are resistors with low Ohms larger?

# Control systems, PID
PID controllers is a very simple, but powerful tool in control systems and automation. It is used for keeping a steady speed using the the cruise-control of your car, and it is used for controlling the power of quadcopter propellers. The modern day PID controller is as old as 1922 when it was developed for automatic control of US navy ships (https://en.wikipedia.org/wiki/PID_controller)

Hopefully this slide helps a little bit:
![](images/PID.jpg)

For my HELIOS project I worked together with Jonas at DESY on a "coldbox". The coldbox uses peltier elements to thermally cycles new ATLAS detector modules inorder to quality test them. The code below was largely copied from what I developed during that project. Now these boxes and its software has been shipped to 14 other ATLAS institutes around the world where thousands of modules will be cycled for the coming years.

![](images/coldbox.jpg)

Here is a great introduction series which I used to develop the PID controller that I used on my project: https://www.youtube.com/watch?v=wkfEZmsQqiA&t=7s

As usual, we start by importing the necessary modules 

In [ ]:
# for this section we import all the modules only here. Make sure you run this cell before running the ones below
import RPi.GPIO as GPIO
import time
import sys
from jupyterplot import ProgressPlot
import numpy as np

Define initial parameters like target temperature, target time, temperature sensor object, kp & ki & kd cosntants etc.

In [ ]:
# Setup GPIO18 for PWM control of the power through the resistor
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)
pi_pwm = GPIO.PWM(18,1000)#create PWM instance with frequency

# Define limits of our output
duty_upper_limit = 100
duty_lower_limit = 0

# Set current time in seconds, and set the time limit
current_time=0
target_time=60*5
time_list=[]
time_list.append(0)

# Define target temperature
target_temperature = 40

# Create a temperature sensor object
sensor_1 = NTC(0,5.21)

# Create an empty list to hold temperature values
temperature_list=[]

# PID part

# Proportional
kp = 15 # Constant for the proportinal part

# Interal
ki = 0.4 # Constant for the integral part
integral_list=[]    # List to hold difference between current temperature and target temperature
                    #The sum of this list is the integral
saturated = True   # If the duty cycle is larger than 100 or less than 0 our system is saturated.
                    # And the integral stops integrating. This is necessary to prevent "Integral wind-up"

# Derivative 
kd = 20 # Constant for the derivative part

Read the comments ant try to understand what happens using the slide above. Feel free to ask me any questions.

**WARNNG!** There is a high probability that this code fails shortly after execution due to some linear fit issue with the derivative part. When it does so, the code will exit **Without turning off power**. When this happens, just executethis cell:

In [ ]:
pi_pwm.stop()

The code below here starts the PID controller

In [ ]:
# Initialise plotting
pp = ProgressPlot(plot_names=['Temperature', 'Proportional', 'Integral', 'Derivative'], x_lim=[0, 60*5], y_lim=[[25, 60],[0, 100],[0, 100],[0, 100]])

integral_list=[] # Make sure integral list is empty if only this cell is run
pi_pwm.start(0)#start PWM of required Duty Cycle 
iterator = 0 # Iterator used to keep derivative fit range small

time_list = []
temperature_list = []

# We put the loop in a try/except block so that if you cancel prematurely,
# the program still exits gracefully without leaving the hot resistor on
try:
    #While loopf for as long as the current time is less than the target
    while current_time<target_time:
        t0 = time.monotonic() # Start a stop watch to see how long one loop takes
        
        # Read the current temperature and append to a list
        # This list is used to calculate the derivative
        current_temperature = sensor_1.read_temperature() 
        temperature_list.append(current_temperature)
        
        # Calcualte the difference between the target temperatrue and current temperature.
        # Often called the "error"
        temperature_difference = target_temperature - current_temperature
        
        # Proportional
        # The proportional part is just the error times the factor kp
        P = temperature_difference * kp
        
        # Intergral
        # The integral part is only the integral of all errros so far
        # Since we use regular 1 second integra, we can simply use the sum
        if saturated != True: # If the output is saturated, we do not att to the integral
            integral_list.append(temperature_difference)
        integral = sum(integral_list)
        I = integral * ki
        
        # Derivative
        # The derivative part is the slope of the temperature times kd
        try:
            slope = np.polyfit(time_list[iterator-7:iterator], temperature_list[iterator-7:iterator], 1)[0]
            D = slope * kd
        except TypeError:
            D=0
        except ValueError:
            D=0
        
        # Propose an output value which is the sum of P and I minus D. Because we want D
        # To counteract large rapid changes
        proposed_duty = P + I - D
        
        # Check if value is within physical limits, if not set saturated to true to stop the integral.
        if proposed_duty > duty_upper_limit:
            saturated = True
            applied_duty = duty_upper_limit
        elif proposed_duty < duty_lower_limit:
            saturated = True
            applied_duty = duty_lower_limit
        else:
            saturated = False
            applied_duty = proposed_duty

        # Apply the proposed or modified proposed value at the limits
        pi_pwm.ChangeDutyCycle(applied_duty)
        
        # Update plots
        pp.update([[current_temperature],[P],[I],[D]])

        # Calculate the time taken for everything and add what is needed for it to take 1 second to run
        t1 = time.monotonic()
        dt = t1 - t0
        sleep_time = 1 - dt
        time.sleep(sleep_time)
        
        # Append current time
        current_time = current_time + dt +sleep_time
        time_list.append(current_time)
        
        # Iterator used to keep derivative fit range small
        iterator +=1
            
# Exit gracefully
except KeyboardInterrupt or ValueError:
    pi_pwm.stop()
    pp.finalize()
    sys.exit()

pi_pwm.stop()
pp.finalize()
sys.exit()


**Exercise**: Look at ``proposed_duty = P + I - D`` in the code above. Comment out I and D and run the code again. How does the PID controller behave?\
**Exercise**: Add back the Integral part, what changes?\
**Exercise**: Can you describe integra wind-up?\
**Exercise**: Add back the Derivative, what changes?\
**Exercise**: How could you get rid of the spike in the Derivative?\
**Exercise**: Try changing the kp, ki, and kd parameters to "Tune" your PID controller to reach the target temperature faster. Did you succeed?\
**Exercise**: You can read about a heuristic method of tuning a PID controller here:https://en.wikipedia.org/wiki/Ziegler%E2%80%93Nichols_method
